In [31]:
import boto3
from fastparquet import ParquetFile
import pandas as pd
import glob
import os
import random
import numpy as np
from scipy import sparse

## Sampling
random select 1 percent samples from every file with seed = 66. 
On average 5 to 6 second for each file. 
each file contains around 60000 records. total sample size = 60000 * 200

In [78]:
file_num = 0
res = []
for filepath in glob.glob(os.path.join('/adobe/data/', '*.snappy.parquet')):
    df = pd.read_parquet(filepath, engine="fastparquet")
    sample = df.sample(frac=0.01, replace=False, random_state=66)
    sample['from_file'] = file_num
    file_num += 1
    res.append(sample)

In [ ]:
df_sample = pd.concat(res)
# df_sample.to_csv('sample.csv')

In [79]:
res[0].head()

,uuid,encodedTraits.type,encodedTraits.size,encodedTraits.indices,encodedTraits.values,from_file
287463,6b86365a-981f-5ff3-976f-e0ac751c7462,0,5197,[800],[1.0],0
530642,c64eb069-0730-5e5a-bd90-0a6dd7f72ef2,0,5197,[800],[1.0],0
320119,77b0c22e-2bd0-5a34-bb73-e815c5c6ea1d,0,5197,[800],[1.0],0
304398,71d61379-f871-5244-8146-d45d919fee1f,0,5197,"[800, 2268, 4199, 4265, 4284, 4413, 4929]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",0
376875,8cfce892-a06f-5dae-8b05-fe8d3c469a82,0,5197,"[800, 4265, 4283, 4413, 4855, 5186]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",0


In [12]:
res[1].head()

,uuid,encodedTraits.type,encodedTraits.size,encodedTraits.indices,encodedTraits.values,from_file
275998,6721ff5f-7795-53c6-8601-fe3cc50805c7,0,5197,"[800, 4413, 4995]","[1.0, 1.0, 1.0]",1
299461,6fea10eb-8f9e-5805-9db1-829780699302,0,5197,[800],[1.0],1
46414,113b203f-a4d2-5c91-bd7d-abd6b623cf84,0,5197,"[800, 4197, 4200, 4265, 4269, 4413, 4924, 5100...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1
458657,ab10a381-2c93-5705-b4d4-ec7f64a87d10,0,5197,"[800, 4393, 4412, 4413, 5051]","[1.0, 1.0, 1.0, 1.0, 1.0]",1
176604,41da5ec2-68ce-5204-ab7b-971e8d2f6836,0,5197,"[800, 4413, 4904, 4975, 5048, 5051]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",1


In [13]:
res[2].head()

,uuid,encodedTraits.type,encodedTraits.size,encodedTraits.indices,encodedTraits.values,from_file
518462,c16f786f-2c36-585c-bc48-47a277fc372f,0,5197,[800],[1.0],2
178872,42971d20-1be5-5725-aa51-5f53e65b6acd,0,5197,[800],[1.0],2
41747,0f97f250-6acb-5cf0-8950-eee1bb95198b,0,5197,[800],[1.0],2
331203,7baa2b24-67ca-58fe-a5c5-beedd817e77c,0,5197,"[800, 2268, 4199, 4265, 4269, 4412, 4413, 5051]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",2
230387,55d5ff64-5126-554b-8ad8-d1229baeaa36,0,5197,"[800, 2268, 4199, 4265, 4269, 4413, 5049]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",2


## Convert dataframe to matrix
the following function takes a dataframe and transforms it to user-traits matrix in sparse format. 

In [73]:
def form_matrix(df):
    M, N = df.shape[0], 5197
    d = res[0].set_index('uuid')['encodedTraits.indices'].to_dict()
    matrix = sparse.lil_matrix((M, N))
    row_names = np.array(list(d.keys()))
    for k in d:
        row_idx, = np.where(row_names == k)
        for v in d[k]:
            matrix[row_idx[0], v] = 1
    return matrix

In [74]:
matrix = form_matrix(res[0])

In [75]:
matrix

<6855x5197 sparse matrix of type '<class 'numpy.float64'>'
	with 65220 stored elements in LInked List format>